# Quality control of WEAVE RSS files

Relative wavelength calibration and throughput test for individual fibres

Based on sky emission lines identified in Row-stacked spectra (RSS)

# 0. Initialisation

## Imports

In [ ]:
%matplotlib ipympl
from matplotlib import pyplot as plt
from matplotlib import colors
from matplotlib.ticker import AutoMinorLocator

import numpy as np
import os
import glob
from time import time
from scipy import ndimage

from astropy.io import fits
from astropy.wcs import WCS
from astropy.coordinates import SkyCoord
from astropy import units as u
from astropy import constants as c
from astropy.table import Table


Plotting functions:

In [ ]:
def new_figure(fig_name, figsize=None, nrows=1, ncols=1, sharex='col', sharey='row', gridspec_kw={'hspace': 0, 'wspace': 0}):
    if figsize is None:
        figsize = (9 + ncols, 4 + nrows)
        
    plt.close(fig_name)
    fig = plt.figure(fig_name, figsize=figsize)
    axes = fig.subplots(nrows=nrows, ncols=ncols, squeeze=False,
                        sharex=sharex, sharey=sharey,
                        gridspec_kw=gridspec_kw
                       )
    fig.set_tight_layout(True)
    for ax in axes.flat:
        ax.xaxis.set_minor_locator(AutoMinorLocator())
        ax.yaxis.set_minor_locator(AutoMinorLocator())
        ax.tick_params(which='both', bottom=True, top=True, left=True, right=True)
        ax.tick_params(which='major', direction='inout', length=8, grid_alpha=.3)
        ax.tick_params(which='minor', direction='in', length=2, grid_alpha=.1)
        ax.grid(True, which='both')

    #fig.suptitle(f'{self.filename} {fig_name}')
    fig.suptitle(fig_name)
    
    return fig, axes


In [ ]:
default_cmap = plt.get_cmap("gist_earth").copy()
default_cmap.set_bad('gray')


def colour_map(fig, ax, cblabel, data, cmap=default_cmap, norm=None, xlabel=None, x=None, ylabel=None, y=None):
    
    if norm is None:
        percentiles = np.array([1, 16, 50, 84, 99])
        ticks = np.nanpercentile(data, percentiles)
        linthresh = np.median(data[data > 0])
        norm = colors.SymLogNorm(vmin=ticks[0], vmax=ticks[-1], linthresh=linthresh)
    else:
        ticks = None
    if y is None:
        y = np.arange(data.shape[0])
    if x is None:
        x = np.arange(data.shape[1])

    im = ax.imshow(data,
                   extent=(x[0]-(x[1]-x[0])/2, x[-1]+(x[-1]-x[-2])/2, y[0]-(y[1]-y[0])/2, y[-1]+(y[-1]-y[-2])/2),
                   interpolation='nearest', origin='lower',
                   cmap=cmap,
                   norm=norm,
                  )
    ax.set_aspect('auto')
    if xlabel is not None:
        ax.set_xlabel(xlabel)
    if ylabel is not None:
        ax.set_ylabel(ylabel)

    cb = fig.colorbar(im, ax=ax, orientation='vertical', shrink=.9)
    cb.ax.set_ylabel(cblabel)
    if ticks is not None:
        cb.ax.set_yticks(ticks=ticks, labels=[f'{value:.3g} ({percent}%)' for value, percent in zip(ticks, percentiles)])
    cb.ax.tick_params(labelsize='small')
    
    return im, cb


## Ancillary data

UVES sky emission atlas: <https://www.eso.org/observing/dfo/quality/UVES/pipeline/sky_spectrum.html>

In [ ]:
wave_flux = np.empty((0, 2))
filenames = glob.glob('sky/UVES_sky_emission_atlas/gident_*.dat')
filenames.sort()
for filename in filenames:
    print(filename)
    wave_flux = np.concatenate((wave_flux, np.loadtxt(filename, usecols=(1, 4), skiprows=3, comments=['#', '--------'])), axis=0)
UVES_atlas = Table(wave_flux, names=('wavelength', 'flux'))

In [ ]:
UVES_atlas

## RSS files

In [ ]:
def find_lower_envelope(x, y, min_separation):
    '''
    Fit lower envelope of a single spectrum:
    1) Find local minima, with a minimum separation `min_separation`.
    2) Interpolate linearly between them.
    '''
    valleys = []
    y[np.isnan(y)] = np.inf
    for i in range(min_separation, y.size-min_separation-1):
        if np.argmin(y[i-min_separation:i+min_separation+1]) == min_separation:
            valleys.append(i)
    y[~np.isfinite(y)] = np.nan
    return np.fmin(y, np.interp(x, x[valleys], y[valleys]))


def boxcar(data, h):
    cumulative = np.nancumsum(data)
    left = np.clip(np.arange(data.size) - h, 0, data.size - 1)
    right = np.clip(np.arange(data.size) + h, 0, data.size - 1)
    return (cumulative[right] - cumulative[left]) / (right - left)


def find_continuum(x, y, h):
    continuum = find_lower_envelope(x, y, h)
    #continuum = boxcar(np.fmin(continuum, boxcar(continuum, h)), h)

    offset = y - continuum
    offset = offset[offset < 2*np.nanmedian(offset)]
    ##continuum += .5*np.nanmedian(offset)
    continuum += np.nanpercentile(offset, 50)
    #continuum = find_lower_envelope(x, continuum, h)

    #return lower_envelope_original + np.nanmedian(offset)
    #continuum = boxcar(lower_envelope_original + np.nanmedian(offset), 10*h)
    #continuum = boxcar(spectrum, h)
    #continuum = find_lower_envelope(x, continuum, h)
    #continuum = boxcar(continuum, 2*h)
    #continuum = np.fmax(lower_envelope_original, continuum)
    
    return continuum


In [ ]:
class Emission_line(object):
    
    def __init__(self, left=0, right=-1):
        '''Simple class to handle emission lines (object or sky)'''
        self.blue_index = left
        self.red_index = right
        self.observed_wavelength = None
        self.reference_wavelength = None
        self.mean_intensity = None
        self.reference_intensity = None
        

In [ ]:
class WEAVE_RSS(object):
    
    def __init__(self, filename):
        '''Read a WEAVE "single exposure" file (i.e. row-stacked spectra for just one arm)'''
        self.filename = filename
        self.hdu = fits.open(filename)
        self.wcs = WCS(self.hdu[1].header)
        pixels = np.arange(self.hdu[1].data.shape[1])
        self.wavelength = self.wcs.spectral.array_index_to_world(pixels).to_value(u.Angstrom)
        self.counts = self.hdu[3].data
        self.counts_error = np.where(self.hdu[4].data > 0, 1/np.sqrt(self.hdu[4].data), np.nan)
        self.sky_counts = self.hdu[3].data - self.hdu[1].data
        self.sensitivity_function = self.hdu[5].data
        self.flux = self.hdu[1].data*self.sensitivity_function
        self.sky_sub_ivar = self.hdu[2].data
        
        bad = np.isnan(self.counts_error).nonzero()
        self.counts[bad] = np.nan
        self.sky_counts[bad] = np.nan
        self.sensitivity_function[bad] = np.nan
        self.flux[bad] = np.nan
        self.sky_sub_ivar[bad] = np.nan
        
        self.fibtable = self.hdu[6].data
        #self.sky_fibres = np.where(self.fibtable['TARGUSE'] != 'Patata')
        self.sky_fibres = np.where(self.fibtable['TARGUSE'] == 'S')
        self.target_fibres = np.where(self.fibtable['TARGUSE'] == 'T')
        self.n_sky_fibres = self.sky_fibres[0].size
        self.n_fibres = self.counts.shape[0]

        heliocentric_correction = (1 + np.nanmean(self.fibtable['Helio_cor'])/3e5)
        wave = UVES_atlas['wavelength'] * heliocentric_correction
        inside = np.where((wave > self.wavelength[0]) & (wave < self.wavelength[-1]))
        self.sky_lines = UVES_atlas[inside].copy()
        self.sky_lines['wavelength'] *= heliocentric_correction
        
        self.intensity = self.counts.copy()
        self.continuum, self.strong_sky_lines = self.find_sky_lines()
        self.f_sky = self.estimate_sky()
        self.line_offset, self.line_throughput = self.trace_sky_lines()

        self.original_offset = self.line_offset.copy()
        self.original_throughput = self.line_throughput.copy()
        self.original_continuum = self.continuum.copy()

        for i in range(0):
            line_weight = self.strong_sky_lines['counts']
            total_weight = np.nansum(line_weight)
            # TODO: flux-conserving interpolation
            wavelength_correction = np.nansum(self.line_offset*line_weight[:, np.newaxis], axis=0) / total_weight
            #wavelength_correction /= 2
            for fibre in range(self.n_fibres):
                self.intensity[fibre] = np.interp(self.wavelength,
                                                         self.wavelength - wavelength_correction[fibre], self.intensity[fibre])        
                self.continuum[fibre] = np.interp(self.wavelength,
                                                         self.wavelength - wavelength_correction[fibre], self.continuum[fibre])        
            fibre_throughput = np.nansum(self.line_throughput*line_weight[:, np.newaxis], axis=0) / total_weight
            self.intensity /= fibre_throughput[:, np.newaxis]
            fluxcal = np.nanmean(self.counts) / np.nanmean(self.intensity)
            print(fluxcal, fibre_throughput[302])
            self.intensity *= fluxcal
            self.continuum *= fluxcal
            self.continuum /= fibre_throughput[:, np.newaxis]
            self.f_sky = self.estimate_sky()
            self.line_offset, self.line_throughput = self.trace_sky_lines()

        for i in range(2):
            # TODO: flux-conserving interpolation
            wavelength_correction = np.nanmedian(self.line_offset, axis=0)
            for fibre in range(self.n_fibres):
                self.intensity[fibre] = np.interp(self.wavelength,
                                                         self.wavelength - wavelength_correction[fibre], self.intensity[fibre])        
                self.continuum[fibre] = np.interp(self.wavelength,
                                                         self.wavelength - wavelength_correction[fibre], self.continuum[fibre])        
            fibre_throughput = np.nanmedian(self.line_throughput, axis=0)
            self.intensity /= fibre_throughput[:, np.newaxis]
            fluxcal = np.nanmean(self.counts) / np.nanmean(self.intensity)
            print(fluxcal, fibre_throughput[302])
            self.intensity *= fluxcal
            self.continuum *= fluxcal
            self.continuum /= fibre_throughput[:, np.newaxis]
            self.f_sky = self.estimate_sky()
            self.line_offset, self.line_throughput = self.trace_sky_lines()


    def find_sky_lines(self, min_separation=5):
        '''Find continuum and identify strong sky emission lines'''
        
        # Find continuum for all fibres:
        print(f"> Find continuum for {self.n_fibres} fibres:")
        t0 = time()
        continuum = np.empty_like(self.counts)
        for i, spectrum in enumerate(self.intensity):
            continuum[i] = find_continuum(self.wavelength, spectrum, min_separation)
        print(f"  Done ({time()-t0:.3g} s)")

        # Identify strong sky emission lines:
        #line_fraction = (self.intensity - continuum) / (self.intensity + continuum)
        line_fraction = 1 - continuum/self.intensity
        p16, p50 = np.nanpercentile(line_fraction, [16, 50])
        line_threshold = p50 + 3*(p50-p16)

        #line_mask = np.all(line_fraction > line_threshold, axis=0)
        line_mask = np.nanmedian(line_fraction, axis=0) > line_threshold
        line_mask[0] = False
        line_mask[-1] = False

        line_left = np.where(~line_mask[:-1] & line_mask[1:])[0]
        line_right = np.where(line_mask[:-1] & ~line_mask[1:])[0]
        line_right += 1
        print(f'  {line_left.size} strong sky lines ({np.count_nonzero(line_mask)} out of {self.wavelength.size} wavelengths with line fraction > {line_threshold:.3f})')
        min_line_width = int(2 * np.nanpercentile(line_right-line_left, 50))
        line_left = np.fmin(line_left, (line_left + line_right - min_line_width + 1) // 2).clip(0, self.wavelength.size)
        line_right = np.fmax(line_right, (line_left + line_right + min_line_width + 1) // 2).clip(0, self.wavelength.size)

        t = Table((line_left, line_right), names=('left', 'right'))
        t.add_column(0., name='wavelength')
        t.add_column(0., name='counts')
        t.add_column(0., name='reference_wavelength')
        t.add_column(0., name='reference_flux')
        for line in t:
            wavelength = self.wavelength[line['left']:line['right']]
            inside = np.where((self.sky_lines['wavelength'] >= wavelength[0])
                              & (self.sky_lines['wavelength'] < wavelength[-1]))
            line['reference_flux'] = np.nansum(self.sky_lines[inside]['flux'])
            line['reference_wavelength'] = np.nansum(self.sky_lines[inside]['flux'] * self.sky_lines[inside]['wavelength'])
            line['reference_wavelength'] /= line['reference_flux']
        return continuum, t[t['reference_flux'] > 0]
        
        
    def estimate_sky(self, max_bins=101):
        '''Linear fit at every wavelength for the spaxels with fainter flux'''
        print(f"> Estimating sky:")
        t0 = time()

        flux = np.nanmean(self.intensity, axis=1)
        flux -= np.nanmin(flux)
        faint = np.where(flux < 2*np.nanmedian(flux))[0]

        f_sky = np.zeros_like(self.wavelength)
        for i in range(self.wavelength.size):
            valid = np.isfinite(self.intensity[faint, i])
            if np.count_nonzero(valid):
                slope, f_sky[i] = np.polyfit(flux[faint][valid], self.intensity[faint, i][valid], 1)
                if np.isnan(f_sky[i]) or slope < 0:
                    f_sky[i] = np.nanmean(self.intensity[faint, i])
            else:
                f_sky[i] = np.nan

        print(f"  Done ({time()-t0:.3g} s)")
        return f_sky


    def trace_sky_lines(self, min_separation=5):
        '''Trace the wavelength and intensity of sky emission lines'''

        reference_spectrum = self.f_sky.copy()
        reference_spectrum -= find_continuum(self.wavelength, reference_spectrum, min_separation)
        
        for line in self.strong_sky_lines:
            wavelength = self.wavelength[line['left']:line['right']]
            spectrum = reference_spectrum[line['left']:line['right']]
            weight = spectrum
            #weight = spectrum**2
            #weight = spectrum**3
            line['wavelength'] = np.nansum(weight*wavelength) / np.nansum(weight)
            line['counts'] = np.nanmean(spectrum)
            #line['counts'] = np.nanstd(spectrum)

        #line_weight = self.strong_sky_lines['counts']
        #total_weight = np.nansum(line_weight)
        
        # Trace line wavelengths for every fibre to compare with the reference spectrum:

        line_fibre_wavelength = []
        line_fibre_intensity = []
        for line in self.strong_sky_lines:
            left = line['left']
            right = line['right']
            weight = (self.intensity[:, left:right] - self.continuum[:, left:right])
            #weight = self.intensity[:, left:right]**2
            line_fibre_wavelength.append(np.nansum(weight * self.wavelength[np.newaxis, left:right], axis=1) / np.nansum(weight, axis=1))
            line_fibre_intensity.append(np.nanmean(self.intensity[:, left:right] - self.continuum[:, left:right], axis=1))
            #line_fibre_intensity.append(np.nanstd(self.intensity[:, left:right] - self.continuum[:, left:right], axis=1))
            
        line_offset = np.array(line_fibre_wavelength) - self.strong_sky_lines['wavelength'][:, np.newaxis]
        line_offset -= np.nanmedian(line_offset, axis=1)[:, np.newaxis]
        #line_offset -= np.nanmean(line_offset, axis=1)[:, np.newaxis]
        #line_offset -= (np.nansum(line_offset*line_weight[:, np.newaxis], axis=1)/total_weight)[:, np.newaxis]

        line_throughput = np.array(line_fibre_intensity) / self.strong_sky_lines['counts'][:, np.newaxis]
        line_throughput /= np.nanmedian(line_throughput)
        #line_throughput /= np.nanmedian(line_throughput, axis=1)[:, np.newaxis]
        #line_throughput /= np.nanmean(line_throughput, axis=1)[:, np.newaxis]
        #line_throughput /= (np.nansum(line_throughput*line_weight[:, np.newaxis], axis=1)/total_weight)[:, np.newaxis]
        
        return line_offset, line_throughput
        
        
#rss = WEAVE_RSS('data/v3/casu.ast.cam.ac.uk/weavedata/1500016316/L1/single_3039342.fit')
#rss = WEAVE_RSS('data/v3/casu.ast.cam.ac.uk/weavedata/1500016658/L1/single_3042890.fit')
rss = WEAVE_RSS('data/v3/casu.ast.cam.ac.uk/weavedata/1500016520/L1/single_3045973.fit')


# QC tests

## Wavelength correction

In [ ]:
def test_relative_wavelength_correction(self):
    '''Relative line offset, before and after correction'''
    fig, axes = new_figure('relative_wavelength_correction', nrows=4)
    
    
    ax = axes[0, 0]
    ax.set_ylabel('original $\Delta\lambda$ [$\AA$]')
    ax.set_ylim(-.25, .25)

    p16, p50, p84 = np.nanpercentile(self.original_offset, [16, 50, 84], axis=0)
    ax.plot(p50, 'k-', alpha=.5)
    ax.fill_between(np.arange(self.n_fibres), p16, p84, color='k', alpha=.1)
    
    #for line in self.sky_fibres[0]:
    #    ax.axvline(line, c='b', ls='-', alpha=.2)
    cb = fig.colorbar(None, ax=ax)
    cb.remove()

    
    ax = axes[1, 0]
    ax.set_ylabel('new $\Delta\lambda$ [$\AA$]')
    ax.set_ylim(-.25, .25)
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    p16, p50, p84 = np.nanpercentile(self.line_offset, [16, 50, 84], axis=0)
    ax.plot(p50, 'k-', alpha=.5)
    ax.fill_between(np.arange(self.n_fibres), p16, p84, color='k', alpha=.1)
    cb = fig.colorbar(None, ax=ax)
    cb.remove()

    
    ax = axes[2, 0]
    ax.set_ylim(-1.5, len(self.strong_sky_lines)+.5)
    im, cb = colour_map(fig, ax, 'original $\Delta\lambda$ [$\AA$]', self.original_offset,
                        xlabel='fibre', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=-.2, vmax=.2))

    ax = axes[3, 0]
    ax.set_ylim(-1.5, len(self.strong_sky_lines)+.5)
    ax.get_shared_y_axes().join(ax, axes[2, 0])
    im, cb = colour_map(fig, ax, 'new $\Delta\lambda$ [$\AA$]', self.line_offset,
                        xlabel='fibre', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=-.2, vmax=.2))

    plt.savefig(f'{self.filename}-{fig.get_label()}.pdf')


test_relative_wavelength_correction(rss)

In [ ]:
def test_absolute_wavelength_calibration(self):
    '''Compare measured wavelength of sky lines with UVES atlas'''
    fig, axes = new_figure('absolute_wavelength_calibration', ncols=2)

    
    ax = axes[0, 0]
    ax.set_ylabel(r'$\Delta\lambda\ [\AA]$')
    ax.set_xlabel(r'wavelength [$\AA$]')
    
    ratio = self.strong_sky_lines['wavelength'] - self.strong_sky_lines['reference_wavelength']
    really_strong = np.where(self.strong_sky_lines['reference_flux'] > np.nanmean(self.strong_sky_lines['reference_flux']))
    ax.plot(self.strong_sky_lines['wavelength'][really_strong], ratio[really_strong], 'co')
    ax.plot(self.strong_sky_lines['wavelength'], ratio, 'k+')
    
    #p16_o, p50_o, p84_o = np.nanpercentile(self.line_offset, [16, 50, 84], axis=1)
    #ax.errorbar(self.strong_sky_lines['wavelength'], p50_o, yerr=(np.abs(p16_o), p84_o), fmt='rx')
    #ax.fill_between(self.strong_sky_lines['wavelength'], p16_o, p84_o, color='k', alpha=.1)

    p16, p50, p84 = np.nanpercentile(ratio, [16, 50, 84])
    ax.axhline(p50, c='k', ls=':', alpha=.5)
    ax.fill_between(self.strong_sky_lines['wavelength'], p16, p84, color='k', alpha=.1)

    
    ax = axes[0, 1]
    ax.set_xlabel(r'counts')
    
    ax.plot(self.strong_sky_lines['counts'][really_strong], ratio[really_strong], 'co')
    ax.plot(self.strong_sky_lines['counts'], ratio, 'k+')
    #ax.errorbar(self.strong_sky_lines['counts'], p50_o, yerr=(np.abs(p16_o), p84_o), fmt='rx')

    ax.axhline(p50, c='k', ls=':', alpha=.5, label=f'{p50:.2f}')
    ax.fill_between(np.sort(self.strong_sky_lines['counts']), p16, p84, color='k', alpha=.1, label=f'{p16:.2f} - {p84:.2f}')
    ax.legend()
    
test_absolute_wavelength_calibration(rss)

## Fibre throughput

In [ ]:
def test_fibre_throughput(self):
    '''Fibre throughput, before and after correction'''
    fig, axes = new_figure('fibre_throughput', nrows=4)

    
    ax = axes[0, 0]
    ax.set_ylabel('original throughput')
    ax.set_ylim(.75, 1.25)

    p16, p50, p84 = np.nanpercentile(self.original_throughput, [16, 50, 84], axis=0)
    ax.plot(p50, 'k-', alpha=.5)
    ax.fill_between(np.arange(self.n_fibres), p16, p84, color='k', alpha=.1)
    
    #for line in self.sky_fibres[0]:
    #    ax.axvline(line, c='b', ls='-', alpha=.2)
    cb = fig.colorbar(None, ax=ax)
    cb.remove()

    
    ax = axes[1, 0]
    ax.set_ylabel('new throughput')
    ax.set_ylim(.75, 1.25)
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    p16, p50, p84 = np.nanpercentile(self.line_throughput, [16, 50, 84], axis=0)
    ax.plot(p50, 'k-', alpha=.5)
    ax.fill_between(np.arange(self.n_fibres), p16, p84, color='k', alpha=.1)
    cb = fig.colorbar(None, ax=ax)
    cb.remove()

    
    ax = axes[2, 0]
    ax.set_ylim(-1.5, len(self.strong_sky_lines)+.5)
    im, cb = colour_map(fig, ax, 'original throughput', self.original_throughput,
                        xlabel='fibre', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=.8, vmax=1.2))

    ax = axes[3, 0]
    ax.set_ylim(-1.5, len(self.strong_sky_lines)+.5)
    ax.get_shared_y_axes().join(ax, axes[2, 0])
    im, cb = colour_map(fig, ax, 'new throughput', self.line_throughput,
                        xlabel='fibre', ylabel='line ID (increasing $\lambda$)', cmap='turbo', norm=colors.Normalize(vmin=.8, vmax=1.2))

    plt.savefig(f'{self.filename}-{fig.get_label()}.pdf')


test_fibre_throughput(rss)

In [ ]:
def test_throughput_map(self):
    '''Spatial distribution and dependence on total flux'''
    flux = np.nanmean(self.intensity, axis=1)
    fig, axes = new_figure('throughput_map', nrows=2)
    
    ax = axes[0, 0]
    ax.set_ylabel(r'original throughput')
    ax.set_ylim(.65, 1.55)
    p16, p50, p84 = np.nanpercentile(self.original_throughput, [16, 50, 84], axis=0)
    ax.errorbar(flux, p50, yerr=(p50-p16, p84-p50), fmt='k+', alpha=.2)
    
    ax = axes[1, 0]
    ax.set_ylabel(r'new throughput')
    ax.set_ylim(.65, 1.55)
    #ax.set_ylim(.8, 1.2)
    #ax.set_yscale('log')
    p16, p50, p84 = np.nanpercentile(self.line_throughput, [16, 50, 84], axis=0)
    ax.errorbar(flux, p50, yerr=(p50-p16, p84-p50), fmt='k+', alpha=.2)

    ax.set_xlabel(r'mean intensity')
    ax.set_xscale('log')


test_throughput_map(rss)

In [ ]:
#rss.strong_sky_lines.show_in_notebook()

## Sky subtraction

In [ ]:
def test_sky_spectrum(self):
    '''Compare different methods to estimate the sky spectrum. Show average subtracted spectrum, as well as fibre with maximuum signal.'''
    fig, axes = new_figure('sky_spectrum', nrows=3)#, figsize=(10, 8))

    ax = axes[0, 0]
    ax.set_ylabel('sky counts')
    #ax.set_yscale('log')
    ax.plot(self.wavelength, self.sky_counts[0], 'r-', alpha=.5, label='original sky')
    ax.plot(self.wavelength, self.f_sky, 'k-', alpha=1, label='linear fit')
    ax.legend()

    ax = axes[1, 0]
    ax.set_ylabel('mean spectrum')
    ax.plot(self.wavelength, np.nanmean(self.counts, axis=0) - self.sky_counts[0], 'r-', alpha=.5, label='original mean spectrum')
    ax.plot(self.wavelength, np.nanmean(self.intensity, axis=0) - self.f_sky, 'k-', alpha=1, label='new mean spectrum')

    ax = axes[2, 0]
    peak = np.nanargmax(np.nanmean(self.intensity, axis=1))
    ax.set_ylabel(f'fibre {peak} (peak)')
    ax.plot(self.wavelength, self.counts[peak] - self.sky_counts[0], 'r-', alpha=.5, label=f'fibre {peak} (original)')
    ax.plot(self.wavelength, self.intensity[peak] - self.f_sky, 'k-', alpha=1, label=f'fibre {peak} (new)')

    ax.set_xlabel(r'wavelength [$\AA$]')
    for ax in axes.ravel():
        for line in self.strong_sky_lines:
            ax.axvspan(self.wavelength[line['left']], self.wavelength[line['right']], color='b', alpha=.1)

    plt.savefig(f'{self.filename}-{fig.get_label()}.pdf')


test_sky_spectrum(rss)

In [ ]:
def test_sky_map(self):
    '''Compare different sky estimates'''
    fig, axes = new_figure('sky_map', nrows=5)

    ax = axes[0, 0]
    ax.set_ylabel('fibre')
    im, cb = colour_map(fig, ax, 'original counts', self.counts, xlabel='', x=self.wavelength)

    ax = axes[1, 0]
    ax.set_ylabel('fibre')
    im, cb = colour_map(fig, ax, 'original subtraction', self.counts-self.sky_counts[0], xlabel='', x=self.wavelength, norm=cb.norm)
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    ax = axes[2, 0]
    ax.set_ylabel('fibre')
    im, cb = colour_map(fig, ax, 'new subtraction', self.intensity-self.f_sky, xlabel='', x=self.wavelength, norm=cb.norm)
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    ax = axes[3, 0]
    ax.set_ylabel('fibre')
    im, cb = colour_map(fig, ax, 'new counts', self.intensity, xlabel='', x=self.wavelength, norm=cb.norm)
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    ax = axes[4, 0]
    ax.set_ylabel('fibre')
    im, cb = colour_map(fig, ax, 'ratio', self.intensity/self.counts, xlabel='', x=self.wavelength, cmap='turbo')
    ax.get_shared_y_axes().join(ax, axes[0, 0])

    ax.set_xlabel(r'wavelength [$\AA$]')
    plt.savefig(f'{self.filename}-{fig.get_label()}.pdf')


test_sky_map(rss)

# Illustrative examples

## Line-continuum separation

Use the sky spectrum as an illustrative example:

In [ ]:
spectrum = rss.f_sky
fibre = 497
spectrum = rss.counts[fibre]


#lower_envelope_smooth = boxcar(np.fmax(lower_envelope_original, find_lower_envelope(rss.wavelength, smooth, h)), h)

#continuum = boxcar(np.fmax(lower_envelope_original, find_lower_envelope(rss.wavelength, np.fmax(lower_envelope_original, lower_envelope_smooth), h)), h)
#continuum = (lower_envelope_original + lower_envelope_smooth) / 2

In [ ]:
fig, axes = new_figure('line_continuum_separation')

ax = axes[0, 0]
ax.set_yscale('log')
ax.set_ylim(30, 3e4)

ax.plot(rss.wavelength, spectrum, 'b-', alpha=.2, label='spectrum')
#ax.plot(rss.wavelength, smooth, 'k:', alpha=1, label='boxcar')
#ax.plot(rss.wavelength, lower_envelope_smooth, 'r--', alpha=1, label='lower envelope smooth')
#ax.plot(rss.wavelength, lower_envelope_original, 'k--', alpha=1, label='lower envelope original')
#ax.plot(rss.wavelength, spectrum - continuum, 'k-', alpha=.2, label='line emission')

ax.plot(rss.wavelength, find_continuum(rss.wavelength, spectrum, 5), 'k-', alpha=1, label='continuum')
ax.plot(rss.wavelength, find_lower_envelope(rss.wavelength, spectrum, 5), 'k--', alpha=1, label='lower envelope')

'''
ax.plot(rss.wavelength, rss.counts[fibre], 'r:', alpha=1, label='old intensity')
ax.plot(rss.wavelength, rss.intensity[fibre], 'r-', alpha=1, label='new intensity')
ax.plot(rss.wavelength, rss.original_continuum[fibre], 'r:', alpha=1, label='old continuum')
ax.plot(rss.wavelength, rss.continuum[fibre], 'r--', alpha=1, label='new continuum')
'''

ax.legend()

plt.show()

In [ ]:
fig, axes = new_figure('line_emission_map', nrows=4, sharey=True)

total_cont = np.sum(rss.continuum, axis=1)
x=rss.wavelength
x=np.arange(rss.wavelength.size)

ax = axes[0, 0]
im, cb = colour_map(fig, ax, 'continuum', rss.continuum, x=x)

ax = axes[1, 0]
im, cb = colour_map(fig, ax, 'counts', rss.counts, x=x)

ax = axes[2, 0]
im, cb = colour_map(fig, ax, 'line emission', rss.intensity-rss.continuum, x=x)

ax = axes[3, 0]
im, cb = colour_map(fig, ax, 'line ratio', 1-rss.continuum/rss.intensity, x=x)
for line in rss.strong_sky_lines:
    ax.axvline(line['left'], c='w', ls='--')
    ax.axvline(line['right'], c='k', ls='--')    

## Sky estimation from linear fit

In [ ]:
total_cont = np.nansum(rss.intensity, axis=1)
total_cont -= np.nanmin(total_cont)
f_sky = np.zeros_like(rss.wavelength)
f_obj = np.zeros_like(rss.wavelength)

faint = np.where(total_cont < 2*np.nanmedian(total_cont))[0]
for i in range(rss.wavelength.size):
    valid = np.isfinite(rss.intensity[faint, i])
    if np.count_nonzero(valid):
        f_obj[i], f_sky[i] = np.polyfit(total_cont[faint][valid], rss.intensity[faint, i][valid], 1)
        if np.isnan(f_sky[i]) or f_obj[i] < 0:
            #print(np.count_nonzero(np.isnan(total_cont[faint])), np.count_nonzero(np.isnan(rss.intensity[faint, i])), ' are nan')
            f_sky[i] = np.nanmedian(rss.intensity[faint, i])
            f_obj[i] = 0
    else:
        f_sky[i] = np.nan


In [ ]:
fig, axes = new_figure('linear fit', nrows=1)

ax = axes[0, 0]
pix = 0
ax.plot(total_cont, rss.counts[:, pix], 'c.')
ax.plot(total_cont[faint], rss.counts[faint, pix], 'r.')
ax.plot(total_cont[faint], f_sky[pix] + total_cont[faint]*f_obj[pix], 'k+')

plt.show()
print(rss.wavelength[pix], f_sky[pix])

## Line Spread Function (LSF)

Parameters:

In [ ]:
LSF_requested_resolution = 0.01  # Angstrom
LSF_wavelength_range = 20  # Angstrom
LSF_dlambda = np.arange(-LSF_wavelength_range, LSF_wavelength_range+.5*LSF_requested_resolution, LSF_requested_resolution)

Function definitions:

In [ ]:
def normalise(x):
    x -= np.median(x)
    norm = x[x.size//2]
    if norm > 0:
        x /= norm
    else:
        x *= np.nan
    return x


def find_LSF(delta_l, spectrum):
    median_skyline = np.zeros((rss.sky_lines['wavelength'].size, delta_l.size))
    for i, line in enumerate(rss.sky_lines['wavelength']):
        sed = np.interp(line+delta_l, rss.wavelength, spectrum)
        sed = normalise(sed)
        median_skyline[i] = sed
    return normalise(np.nanmedian(median_skyline, axis=0))


def find_FWHM(delta_l, profile):
    threshold = np.max(profile)/2
    left = np.max(delta_l[(delta_l < 0) & (profile < threshold)])
    right = np.min(delta_l[(delta_l > 0) & (profile < threshold)])
    return right-left


In [ ]:
def gaussian_profile(x, mu=0, sigma=1, norm=True):
    g = np.exp(-.5 * ((x-mu) / sigma)**2)
    if norm:
        g /= np.sqrt(2*np.pi) * sigma
    return  g


def refine_Gaussian(x, I, mu0, sigma0):

    good = np.where(np.isfinite(I))
    weight = np.exp(-.5 * ((x[good]-mu0) / sigma0)**2) * (I[good] - np.min(I[good]))
    total_weight = np.sum(weight)

    ivar0 = 1 / sigma0**2
    mu = np.sum(weight * x[good]) / total_weight
    ivar = total_weight / np.sum(weight * (x[good] - mu)**2)

    ivar1 = ivar - ivar0
    mu1 = (mu*ivar - mu0*ivar0) / ivar1
    return mu1, 1/np.sqrt(ivar1)


def fit_Gaussian(x, I, mu0=None, sigma0=None):

    if mu0 is None:
        mu0 = np.nanmean(x)
    if sigma0 is None:
        sigma0 = np.nanstd(x)
    
    delta2 = np.inf
    while delta2 > LSF_requested_resolution**2:
        mu1, sigma1 = refine_Gaussian(x, I, mu0, sigma0)
        delta2 = (mu1 - mu0)**2 + (sigma1 - sigma0)**2
        mu0 = mu1
        sigma0 = sigma1
        #print(mu0, sigma0)
    
    return mu0, sigma0

FWHM of mode-based sky:

In [ ]:
sky_LSF = find_LSF(LSF_dlambda, rss.f_sky)
sky_FWHM = find_FWHM(LSF_dlambda, sky_LSF)
#LSF_sigma = sky_FWHM / np.sqrt(8*np.log(2))

LSF_mu, LSF_sigma = fit_Gaussian(LSF_dlambda, sky_LSF, 0, sky_FWHM / np.sqrt(8*np.log(2)))

print(f'FWHM of mode-based sky = {sky_FWHM:.3f} (sigma = {sky_FWHM / np.sqrt(8*np.log(2)):.3f}) Angstrom')
print(f'Moments (mu, sigma) = ({LSF_mu:.3f}, {LSF_sigma:.3f}) Angstrom')

In [ ]:
fig, axes = new_figure('sky_LSF')

    
ax = axes[0, 0]
ax.set_ylabel(r'Median Line Spread Function (LSF)')
#ax.set_yscale('log')
#ax.set_ylim(5e-4, 2)


#ax.plot(LSF_dlambda, np.nancumsum(sky_LSF), 'k--', alpha=.5)
#ax.plot(LSF_dlambda, np.nancumsum(sky_LSF*LSF_dlambda)*np.nancumsum(sky_LSF), 'b-', alpha=.5)
ax.plot(LSF_dlambda, sky_LSF, 'k-', alpha=1, label=f'FWHM = {sky_FWHM:.3f} (sigma = {sky_FWHM / np.sqrt(8*np.log(2)):.3f}) $\\AA$ ')
ax.plot(LSF_dlambda, gaussian_profile(LSF_dlambda, LSF_mu, LSF_sigma, False), 'k--', alpha=1, label=f'Gaussian ($\\mu, \\sigma$) = ({LSF_mu:.3f}, {LSF_sigma:.3f}) $\\AA$ ')
ax.axvline(LSF_mu, c='k', ls=':')
ax.legend()

ax = axes[-1, 0]
ax.set_xlabel(r'$(\lambda - \lambda_0)$ [$\AA$]')